# NS_03 Group Project

## Part1

In [15]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import datetime
import sklearn.metrics

In [16]:
# load data_set, and calculate the mean and std of dataset

# transform = transforms.Compose([
#     transforms.Resize((256, 256)),  # 缩放到224 * 224
#     transforms.ToTensor()
# ])

# # 0 -> mask  1-> nonmask  2 -> not a person
# train_dataset = ImageFolder('./data/train', transform=transform)

# # calculate mean of imgs in each RGB channel
# imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
# print(imgs.shape)

# imgs.view(3, -1).mean(dim=1)  # mean of (number of imgs) in each channel
# imgs.view(3, -1).std(dim=1) # std of (number of imgs) in each channel

In [17]:
# load dataset, normarlize it
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 缩放到224 * 224
    # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4990, 0.4567, 0.4188], std=[0.2913, 0.2778, 0.2836]) 
])

# 0 -> mask  1-> nonmask  2 -> not a person
train_dataset = ImageFolder('./data/train', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = ImageFolder('./data/test', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [18]:
# cnn model

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1), # in=3x256x256; out=32x256x256
            nn.ReLU(),
            nn.MaxPool2d(2), # out=32x128x128
            nn.Conv2d(16, 16, kernel_size=3, padding=1), # in=32x128x128, out=16x128x128
            nn.ReLU(),
            nn.MaxPool2d(2), # out=16x64x64
            nn.Conv2d(16, 32, kernel_size=3, padding=1), # in=16x64x64, out=8x64x64
            nn.ReLU(),
            nn.MaxPool2d(2), # out=8x32x32,
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2) # 16x16
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(64*16*16, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(1024, 32),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(32, 3)
        )
    
    def forward(self, x):
        # conv layer
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)
        
        return x


In [19]:
device = (torch.device('cuda') if torch.cuda.is_available() 
         else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


In [20]:
# define train_loop function
def train_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_dataloader: # loop over batches in dataset
            # move data to GPU if available
            imgs = imgs.to(device=device)  
            labels = labels.to(device=device)
            
            outputs = model(imgs)  # feed a batch through our model
            
            loss = loss_fn(outputs, labels)  # computes the loss
            
            optimizer.zero_grad()  # getting rid of the gradients from the last round
            
            loss.backward()  # performs backward step, compute the gradients of all parameters
            
            optimizer.step()  # updates the model
            
            loss_train += loss.item() # sums of losses we saw over the epoch
            
        # print the average loss per batch, in epoch%10 == 0 
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch, loss_train/len(train_loader)
        ))

In [21]:
model = CNN().to(device=device)  # instantiates cnn model

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list   # number of parameters, and their shapes

(16837043,
 [432, 16, 2304, 16, 4608, 32, 18432, 64, 16777216, 1024, 32768, 32, 96, 3])

In [22]:
# perform training

learning_rate = 0.001
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()  # use cross entropy loss function

# call train_loop() function
train_loop(
    n_epochs = 15,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_dataloader
)
torch.save(model, 'my_model2.pkl')

2021-04-21 11:12:08.802592 Epoch 1, Training loss 0.919861558505467
2021-04-21 11:13:03.167747 Epoch 2, Training loss 0.6948304261480059
2021-04-21 11:13:57.517702 Epoch 3, Training loss 0.5822103185313089
2021-04-21 11:14:51.981474 Epoch 4, Training loss 0.5208582090479987
2021-04-21 11:15:46.530771 Epoch 5, Training loss 0.44716879895755224
2021-04-21 11:16:41.004544 Epoch 6, Training loss 0.37106892125947133
2021-04-21 11:17:35.471691 Epoch 7, Training loss 0.34879638935838425
2021-04-21 11:18:29.952544 Epoch 8, Training loss 0.2794153011270932
2021-04-21 11:19:24.332406 Epoch 9, Training loss 0.20160702232803618
2021-04-21 11:20:18.757346 Epoch 10, Training loss 0.16406727422560966
2021-04-21 11:21:13.182073 Epoch 11, Training loss 0.10442464388906955
2021-04-21 11:22:07.634502 Epoch 12, Training loss 0.05369891197021518
2021-04-21 11:23:01.950802 Epoch 13, Training loss 0.04142235051840544
2021-04-21 11:23:56.281954 Epoch 14, Training loss 0.0604403555393219
2021-04-21 11:24:50.74

In [23]:
# define validate function
def validate(model, train_loader, test_loader):
    model.eval()
    # accuracy on training data and test data
    for name, loader in [("train", train_loader), ("test", test_loader)]:
        correct = 0
        total = 0
        total_predicted = []
        total_labels = []
        
        with torch.no_grad(): # do not want gradients here, as we will not want to update parameters
            for imgs, labels in loader:
                # move data to GPU if available
                imgs = imgs.to(device=device)  
                labels = labels.to(device=device)
                total_labels.append(labels)
                
                # feed input to models
                outputs = model(imgs)  
                
                # gives the index of the highest value as output
                _, predicted = torch.max(outputs, dim=1)  
                total_predicted.append(predicted)
                
                # counts the number of example, total is increased by the batch size
                total += labels.shape[0]  
            
                # the prediction and ground truth agree
                correct += int((predicted == labels).sum()) 
                
        total_predicted = torch.hstack(total_predicted).cpu()
        total_labels = torch.hstack(total_labels).cpu()
        
        print("Accuracy {}: {:.2f}".format(name, correct / total))  
        print(sklearn.metrics.classification_report(total_labels, total_predicted))
        print(sklearn.metrics.confusion_matrix(total_labels, total_predicted))

In [24]:
# measuring accuracy
validate(model, train_dataloader, test_dataloader)

Accuracy train: 0.97
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       270
           1       0.93      1.00      0.96       270
           2       0.98      0.96      0.97       576

    accuracy                           0.97      1116
   macro avg       0.96      0.97      0.97      1116
weighted avg       0.97      0.97      0.97      1116

[[257   4   9]
 [  0 269   1]
 [  6  16 554]]
Accuracy test: 0.83
              precision    recall  f1-score   support

           0       0.89      0.68      0.77       123
           1       0.80      0.98      0.88       124
           2       0.83      0.83      0.83       102

    accuracy                           0.83       349
   macro avg       0.84      0.83      0.83       349
weighted avg       0.84      0.83      0.83       349

[[ 84  23  16]
 [  1 122   1]
 [  9   8  85]]


## Part 2